In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stat
import statsmodels.api as sm
import statsmodels.formula.api as smf#, ols
from statsmodels.stats.outliers_influence import variance_inflation_factor
from scipy.stats import ttest_ind, f_oneway
import scipy.stats as stats
import sys
from pathlib import Path
# # Add src directory to the Python path
sys.path.append(str(Path.cwd().parent))
from src.data_analysis.df_dataattribute_analysis import categorize_columns
from src.data_sourcing.import_export_data import get_data, save_data
from src.data_analysis.df_ttest_anova_analysis import run_one_sample_ttest, run_univariate_tests
from src.data_analysis.df_multicollinearity_analysis import compute_vif
from src.data_analysis.df_correlation_analysis import compute_correlations

In [2]:
df = get_data('featured_engineered_aggregations.csv', 'processed_data')
df.head(5)

,ATL_OR_DR,CAMPAIGN_TYPE,CHANNEL,COST,FREE_TRIALS,HOLIDAY_FLAG,SEASON,MONTH,ATL_OR_DR_CAMPAIGN_TYPE,ATL_OR_DR_CAMPAIGN_TYPE_CHANNEL,...,COST_PER_FREE_TRIALS_OVER_AVG_COST_PER_FREE_TRIALS_BY_ATL_OR_DR_MONTH,AVG_FREE_TRIALS_BY_CAMPAIGN_TYPE_MONTH,MEDIAN_FREE_TRIALS_BY_CAMPAIGN_TYPE_MONTH,FREE_TRIALS_OVER_AVG_FREE_TRIALS_BY_CAMPAIGN_TYPE_MONTH,AVG_COST_PER_FREE_TRIALS_BY_CAMPAIGN_TYPE_MONTH,MEDIAN_COST_PER_FREE_TRIALS_BY_CAMPAIGN_TYPE_MONTH,COST_PER_FREE_TRIALS_OVER_AVG_COST_PER_FREE_TRIALS_BY_CAMPAIGN_TYPE_MONTH,NORMALIZE_FREE_TRIAL_BY_MONTH,NORMALIZE_COST_PER_FREE_TRIALS_BY_MONTH,NORMALIZE_COST_BY_FREE_TRIALS_BY_MONTH
0,DR - Direct Response,Title,paid social,7784.31,86401.15,False,Summer,June,DR - Direct Response_Title,DR - Direct Response_Title_paid social,...,0.074378,39965.780229,24130.56,2.161878,0.942693,0.129226,0.095572,2.015642,0.086220,0.701295
1,DR - Direct Response,Title,app,2474.31,2956.74,False,Summer,August,DR - Direct Response_Title,DR - Direct Response_Title_app,...,1.139687,41913.391463,26403.99,0.070544,0.853002,0.107078,0.981049,0.067269,1.023058,0.013036
2,ATL - Above The Line,Title,paid social,10222.82,12513.08,False,Summer,July,ATL - Above The Line_Title,ATL - Above The Line_Title_paid social,...,0.970911,47447.523454,27828.02,0.263725,0.850812,0.101840,0.960225,0.272262,0.984393,0.226908
3,DR - Direct Response,Title,app,49631.87,17207.04,False,Winter,December,DR - Direct Response_Title,DR - Direct Response_Title_app,...,2.688638,46867.404028,28693.37,0.367143,0.871586,0.109911,3.309360,0.362288,2.645155,0.979314
4,ATL - Above The Line,Title,paid social,2081.06,21758.33,False,Summer,August,ATL - Above The Line_Title,ATL - Above The Line_Title_paid social,...,0.109754,41913.391463,26403.99,0.519126,0.853002,0.107078,0.112127,0.495028,0.116928,0.080683


### Multi-collinearity Testing Analysis

In [14]:
# df.columns

# X_df= df.drop(columns=['FREE_TRIALS', 'ATL_OR_DR_CAMPAIGN_TYPE', 'ATL_OR_DR_CAMPAIGN_TYPE_CHANNEL', 'ATL_OR_DR_CAMPAIGN_TYPE_CHANNEL_MONTH',
#                          'ATL_OR_DR_CAMPAIGN_TYPE_MONTH', 'ATL_OR_DR_CHANNEL_MONTH', 'CAMPAIGN_TYPE_CHANNEL_MONTH', 'CAMPAIGN_TYPE_MONTH', 'CHANNEL_MONTH',
#                            'LOG_FREE_TRIALS', 'ATL_OR_DR_MONTH'], axis=1)
# X_df.columns
#
hashmap = categorize_columns(df)
formula = hashmap['numerical']
formula.remove('FREE_TRIALS')
formula.remove('LOG_FREE_TRIALS')
formula.remove('LOG_COST')
formula.extend(['CHANNEL', 'HOLIDAY_FLAG', 'SEASON']) #'CHANNEL'
print("formula:", formula)
temp_df = df[formula]
vif_result = compute_vif(
    data=temp_df,
)
vif_result
# '''
# Try an aggregation calculation by Channel and month? So 2 group by's?
# '''
vif_result[(vif_result['VIF'] > 5) & (vif_result['VIF'] != float('inf'))]

formula: ['COST', 'COST_PER_FREE_TRIALS', 'COST_BY_FREE_TRIALS', 'LOG_COST_PER_FREE_TRIALS', 'LOG_COST_BY_FREE_TRIALS', 'AVG_COST_MONTH', 'MEDIAN_COST_MONTH', 'AVG_COST_CAMPAIGN_TYPE_CHANNEL', 'MEDIAN_COST_CAMPAIGN_TYPE_CHANNEL', 'AVG_FREE_TRIALS_MONTH', 'MEDIAN_FREE_TRIALS_MONTH', 'AVG_FREE_TRIALS_BY_CHANNEL_MONTH', 'MEDIAN_FREE_TRIALS_BY_CHANNEL_MONTH', 'FREE_TRIALS_OVER_AVG_FREE_TRIALS_BY_CHANNEL_MONTH', 'AVG_COST_PER_FREE_TRIALS_BY_CHANNEL_MONTH', 'MEDIAN_COST_PER_FREE_TRIALS_BY_CHANNEL_MONTH', 'COST_PER_FREE_TRIALS_OVER_AVG_COST_PER_FREE_TRIALS_BY_CHANNEL_MONTH', 'AVG_FREE_TRIALS_BY_ATL_OR_DR_MONTH', 'MEDIAN_FREE_TRIALS_BY_ATL_OR_DR_MONTH', 'FREE_TRIALS_OVER_AVG_FREE_TRIALS_BY_ATL_OR_DR_MONTH', 'AVG_COST_PER_FREE_TRIALS_BY_ATL_OR_DR_MONTH', 'MEDIAN_COST_PER_FREE_TRIALS_BY_ATL_OR_DR_MONTH', 'COST_PER_FREE_TRIALS_OVER_AVG_COST_PER_FREE_TRIALS_BY_ATL_OR_DR_MONTH', 'AVG_FREE_TRIALS_BY_CAMPAIGN_TYPE_MONTH', 'MEDIAN_FREE_TRIALS_BY_CAMPAIGN_TYPE_MONTH', 'FREE_TRIALS_OVER_AVG_FREE_TRIALS_

,Feature,VIF
0,NORMALIZE_FREE_TRIAL_BY_MONTH,831.269109
1,FREE_TRIALS_OVER_AVG_FREE_TRIALS_BY_ATL_OR_DR_...,655.450760
2,FREE_TRIALS_OVER_AVG_FREE_TRIALS_BY_CAMPAIGN_T...,314.766740
3,NORMALIZE_COST_PER_FREE_TRIALS_BY_MONTH,140.603135
4,COST_PER_FREE_TRIALS_OVER_AVG_COST_PER_FREE_TR...,79.984258
5,COST_PER_FREE_TRIALS_OVER_AVG_COST_PER_FREE_TR...,65.711657
6,COST_PER_FREE_TRIALS,41.091920
7,FREE_TRIALS_OVER_AVG_FREE_TRIALS_BY_CHANNEL_MONTH,38.626175
8,NORMALIZE_COST_BY_FREE_TRIALS_BY_MONTH,33.029209
9,COST_BY_FREE_TRIALS,32.768695


### One Sample T-Test Aanalysis

In [15]:
categorical_list = ['ATL_OR_DR','HOLIDAY_FLAG']
quantitative = ["FREE_TRIALS", "LOG_FREE_TRIALS"]
avg_free_trials = round(df['FREE_TRIALS'].mean(),2)
avg_log_free_trials = round(df['LOG_FREE_TRIALS'].mean(),2)
# 2) Using a dictionary for different hypothesized means per column:
results_2 = run_one_sample_ttest(
    df=df,
    categorical_cols=categorical_list,
    quantitative_cols=quantitative,
    popmean={"FREE_TRIALS": avg_free_trials, "LOG_FREE_TRIALS": avg_log_free_trials},  # different popmeans
    stat_type="mean",
    alpha=0.05,  # more stringent significance level
    sort_results=True
)
results_2

,Categorical Variable,Category,Quantitative Variable,Sample Size,Sample Mean,T-statistic,p-value,Decision,Error
0,HOLIDAY_FLAG,True,LOG_FREE_TRIALS,534,10.201621,-0.637220,0.524255,Fail to Reject H0,None
1,ATL_OR_DR,ATL - Above The Line,LOG_FREE_TRIALS,8601,10.228078,-0.178557,0.858290,Fail to Reject H0,None
2,ATL_OR_DR,DR - Direct Response,LOG_FREE_TRIALS,5507,10.227902,-0.155741,0.876243,Fail to Reject H0,None
3,ATL_OR_DR,DR - Direct Response,FREE_TRIALS,5507,45547.216388,0.112888,0.910123,Fail to Reject H0,None
4,HOLIDAY_FLAG,False,LOG_FREE_TRIALS,13574,10.229048,-0.111226,0.911438,Fail to Reject H0,None
5,ATL_OR_DR,ATL - Above The Line,FREE_TRIALS,8601,45401.434358,-0.085614,0.931775,Fail to Reject H0,None
6,HOLIDAY_FLAG,True,FREE_TRIALS,534,45484.437959,0.009366,0.992530,Fail to Reject H0,None
7,HOLIDAY_FLAG,False,FREE_TRIALS,13574,45457.313076,-0.001986,0.998415,Fail to Reject H0,None


### ANOVA Testing Analysis

In [16]:
# https://towardsdatascience.com/anova-test-with-python-cfbf4013328b
hashmap = categorize_columns(df)
categorical = hashmap['categorical']
categorical.remove('ATL_OR_DR')
categorical.remove('HOLIDAY_FLAG')
categorical

['CAMPAIGN_TYPE',
 'CHANNEL',
 'SEASON',
 'MONTH',
 'ATL_OR_DR_CAMPAIGN_TYPE',
 'ATL_OR_DR_CAMPAIGN_TYPE_CHANNEL',
 'ATL_OR_DR_CAMPAIGN_TYPE_CHANNEL_MONTH',
 'ATL_OR_DR_CAMPAIGN_TYPE_MONTH',
 'ATL_OR_DR_CHANNEL',
 'ATL_OR_DR_CHANNEL_MONTH',
 'ATL_OR_DR_MONTH',
 'CAMPAIGN_TYPE_CHANNEL',
 'CAMPAIGN_TYPE_CHANNEL_MONTH',
 'CAMPAIGN_TYPE_MONTH',
 'CHANNEL_MONTH']

In [17]:
quantitative = ["FREE_TRIALS", "LOG_FREE_TRIALS"]

anova_results_df = run_univariate_tests(
    df=df,
    categorical_cols=categorical,
    quantitative_cols=quantitative,
    test_type="anova"
)
anova_results_df[anova_results_df['p-value'] <= 0.50]

,Categorical Variable,Quantitative Variable,F-statistic,p-value,Error
0,MONTH,FREE_TRIALS,1.745376,0.057730,None
1,ATL_OR_DR_MONTH,FREE_TRIALS,1.437204,0.080345,None
2,CAMPAIGN_TYPE_MONTH,FREE_TRIALS,1.346628,0.104410,None
3,CHANNEL_MONTH,FREE_TRIALS,1.096844,0.170871,None
4,MONTH,LOG_FREE_TRIALS,1.335306,0.197357,None
5,CAMPAIGN_TYPE_CHANNEL_MONTH,FREE_TRIALS,1.061845,0.206772,None
6,ATL_OR_DR_CAMPAIGN_TYPE_MONTH,FREE_TRIALS,1.112372,0.268880,None
7,ATL_OR_DR_CHANNEL_MONTH,FREE_TRIALS,1.050398,0.285439,None
8,ATL_OR_DR_MONTH,LOG_FREE_TRIALS,1.085444,0.352263,None
9,ATL_OR_DR_CAMPAIGN_TYPE_CHANNEL_MONTH,FREE_TRIALS,1.023266,0.361860,None


### Kruskal Wallis Testing Analysis

In [18]:
kruskal_results_df = run_univariate_tests(
    df=df,
    categorical_cols=categorical,
    quantitative_cols=quantitative,
    test_type="kruskal"
)
kruskal_results_df[kruskal_results_df['p-value'] <= 0.60]

,Categorical Variable,Quantitative Variable,H-statistic,p-value,Error
0,MONTH,FREE_TRIALS,15.421565,0.163999,None
1,MONTH,LOG_FREE_TRIALS,15.421565,0.163999,None
2,ATL_OR_DR_MONTH,LOG_FREE_TRIALS,26.406948,0.282034,None
3,ATL_OR_DR_MONTH,FREE_TRIALS,26.406948,0.282034,None
4,SEASON,FREE_TRIALS,2.744503,0.432717,None
5,SEASON,LOG_FREE_TRIALS,2.744503,0.432717,None
6,CHANNEL_MONTH,LOG_FREE_TRIALS,193.675803,0.493063,None
7,CHANNEL_MONTH,FREE_TRIALS,193.675803,0.493063,None
8,ATL_OR_DR_CHANNEL_MONTH,LOG_FREE_TRIALS,233.572700,0.586903,None
9,ATL_OR_DR_CHANNEL_MONTH,FREE_TRIALS,233.572700,0.586903,None


### Pearson Correlation Testing Analysis

In [19]:
# 1) Pearson correlation (numeric only - only linear relationships)
pearson_results = compute_correlations(
    df=df,
    include_categorical=False,
    cat_cols=None  # Do not encode categorical variables
)
pearson_results[
    (pearson_results['p-value'] < 0.5) &
    (pearson_results['Variable 1'] == 'FREE_TRIALS')
]

,Variable 1,Variable 2,Correlation,p-value,Method,Error
10,FREE_TRIALS,NORMALIZE_FREE_TRIAL_BY_MONTH,0.997894,0.000000e+00,pearson,None
14,FREE_TRIALS,FREE_TRIALS_OVER_AVG_FREE_TRIALS_BY_CAMPAIGN_T...,0.995560,0.000000e+00,pearson,None
18,FREE_TRIALS,FREE_TRIALS_OVER_AVG_FREE_TRIALS_BY_ATL_OR_DR_...,0.996181,0.000000e+00,pearson,None
55,FREE_TRIALS,FREE_TRIALS_OVER_AVG_FREE_TRIALS_BY_CHANNEL_MONTH,0.972843,0.000000e+00,pearson,None
84,FREE_TRIALS,LOG_COST_BY_FREE_TRIALS,0.347763,0.000000e+00,pearson,None
86,FREE_TRIALS,LOG_FREE_TRIALS,0.747446,0.000000e+00,pearson,None
108,FREE_TRIALS,NORMALIZE_COST_BY_FREE_TRIALS_BY_MONTH,0.230571,1.356582e-169,pearson,None
117,FREE_TRIALS,COST_BY_FREE_TRIALS,0.219831,5.792956e-154,pearson,None
123,FREE_TRIALS,LOG_COST_PER_FREE_TRIALS,-0.215401,9.361971e-148,pearson,None
177,FREE_TRIALS,AVG_FREE_TRIALS_BY_CHANNEL_MONTH,0.122735,1.752471e-48,pearson,None


### Spearman Correlation Testing Analysis

In [20]:
# 2) Spearman correlation (with categorical one-hot encoding)
spearman_results = compute_correlations(
    df=df,
    include_categorical=True,  # Encode categorical variables
    drop_first=True,
    cat_cols=categorical
)
spearman_results[((spearman_results['Variable 1'] == 'FREE_TRIALS') | (spearman_results['Variable 1'] == 'LOG_FREE_TRIALS')) & (spearman_results['p-value'] < 0.5)]

,Variable 1,Variable 2,Correlation,p-value,Method,Error
13,LOG_FREE_TRIALS,LOG_COST_PER_FREE_TRIALS,-0.460507,0.000000e+00,spearman,None
14,LOG_FREE_TRIALS,LOG_COST_BY_FREE_TRIALS,0.470630,0.000000e+00,spearman,None
17,LOG_FREE_TRIALS,FREE_TRIALS_OVER_AVG_FREE_TRIALS_BY_CHANNEL_MONTH,0.986150,0.000000e+00,spearman,None
20,LOG_FREE_TRIALS,COST_PER_FREE_TRIALS_OVER_AVG_COST_PER_FREE_TR...,-0.449598,0.000000e+00,spearman,None
43,LOG_FREE_TRIALS,FREE_TRIALS_OVER_AVG_FREE_TRIALS_BY_ATL_OR_DR_...,0.997803,0.000000e+00,spearman,None
67,LOG_FREE_TRIALS,COST_PER_FREE_TRIALS_OVER_AVG_COST_PER_FREE_TR...,-0.458958,0.000000e+00,spearman,None
68,LOG_FREE_TRIALS,FREE_TRIALS_OVER_AVG_FREE_TRIALS_BY_CAMPAIGN_T...,0.997544,0.000000e+00,spearman,None
69,LOG_FREE_TRIALS,COST_PER_FREE_TRIALS_OVER_AVG_COST_PER_FREE_TR...,-0.459360,0.000000e+00,spearman,None
70,LOG_FREE_TRIALS,NORMALIZE_FREE_TRIAL_BY_MONTH,0.998711,0.000000e+00,spearman,None
71,LOG_FREE_TRIALS,NORMALIZE_COST_PER_FREE_TRIALS_BY_MONTH,-0.459935,0.000000e+00,spearman,None


### Random Testing Analysis

In [10]:
# import pandas as pd
# import statsmodels.api as sm
# from statsmodels.formula.api import ols

# # Example: Assuming your data is in a DataFrame called df
# # Replace 'FREE_TRIALS' with your dependent variable
# # Replace other variable names with the actual column names from your dataset

# # Create the OLS formula
# formula = """
# FREE_TRIALS ~ HOLIDAY_FLAG
# + ATL_OR_DR + CAMPAIGN_TYPE + CHANNEL + MONTH
# + ATL_OR_DR_CAMPAIGN_TYPE + ATL_OR_DR_CAMPAIGN_TYPE_CHANNEL
# + ATL_OR_DR_CAMPAIGN_TYPE_CHANNEL_MONTH + ATL_OR_DR_CAMPAIGN_TYPE_MONTH
# + ATL_OR_DR_CHANNEL + ATL_OR_DR_CHANNEL_MONTH + ATL_OR_DR_MONTH
# + CAMPAIGN_TYPE_CHANNEL + CAMPAIGN_TYPE_CHANNEL_MONTH + CAMPAIGN_TYPE_MONTH + CHANNEL_MONTH
# + COST + COST_PER_FREE_TRIALS + COST_BY_FREE_TRIALS
# + LOG_COST + LOG_COST_PER_FREE_TRIALS + LOG_COST_BY_FREE_TRIALS
# + AVG_COST_MONTH + MEDIAN_COST_MONTH
# + AVG_COST_CAMPAIGN_TYPE_CHANNEL + MEDIAN_COST_CAMPAIGN_TYPE_CHANNEL
# + AVG_FREE_TRIALS_MONTH + MEDIAN_FREE_TRIALS_MONTH
# """
# # formula = """
# # FREE_TRIALS ~ HOLIDAY_FLAG
# # + MONTH + CHANNEL + CAMPAIGN_TYPE + ATL_OR_DR
# # + COST + COST_PER_FREE_TRIALS + LOG_COST
# # + MONTH:CHANNEL + CAMPAIGN_TYPE:ATL_OR_DR
# # """
# #formula = "FREE_TRIALS ~ MONTH * ATL_OR_DR + CHANNEL"

# # Fit the OLS model
# # model = ols(formula, data=df).fit()

# import statsmodels.api as sm
# model = sm.GLM(df['FREE_TRIALS'], sm.add_constant(X), family=sm.families.Poisson()).fit()
# print(model.summary())


# # Print the summary of the regression model
# print(model.summary())


In [11]:
# pd.DataFrame(model.summary())

In [12]:
# # # Define a function to plot boxplots and detect outliers based on IQR
# # def plot_outliers(column, ax):
# #     ax.boxplot(data[column], vert=False, patch_artist=True, boxprops=dict(facecolor="lightblue"))
# #     ax.set_title(f'Boxplot of {column}')
# #     ax.set_xlabel(column)

# # # Create boxplots for COST and FREE_TRIALS
# # fig, axes = plt.subplots(1, 2, figsize=(12, 5))
# # plot_outliers('COST', axes[0])
# # plot_outliers('FREE_TRIALS', axes[1])
# # plt.tight_layout()
# # plt.show()

# # Calculate outlier thresholds using IQR
# def calculate_outliers(column):
#     q1 = data[column].quantile(0.25)
#     q3 = data[column].quantile(0.75)
#     iqr = q3 - q1
#     lower_bound = q1 - 1.5 * iqr
#     upper_bound = q3 + 1.5 * iqr
#     outliers = data[(data[column] < lower_bound) | (data[column] > upper_bound)]
#     return lower_bound, upper_bound, outliers

# # Remove outliers based on IQR thresholds for LOG_COST and LOG_FREE_TRIALS
# log_cost_bounds = calculate_outliers('LOG_COST')
# log_free_trials_bounds = calculate_outliers('LOG_FREE_TRIALS')

# # Filter data to exclude outliers in LOG_COST and LOG_FREE_TRIALS
# filtered_log_data = data[
#     (data['LOG_COST'] >= log_cost_bounds[0]) & (data['LOG_COST'] <= log_cost_bounds[1]) &
#     (data['LOG_FREE_TRIALS'] >= log_free_trials_bounds[0]) & (data['LOG_FREE_TRIALS'] <= log_free_trials_bounds[1])
# ]

# # Display the size of the dataset before and after outlier removal based on log variables
# log_filtered_size = filtered_log_data.shape[0]

# log_filtered_size


In [13]:
# from sklearn.preprocessing import StandardScaler

# # Select numerical columns to standardize
# numerical_features = ['LOG_COST', 'LOG_COST_SQ']
# scaler = StandardScaler()

# # Standardize numerical features
# filtered_log_data_encoded[numerical_features] = scaler.fit_transform(
#     filtered_log_data_encoded[numerical_features]
# )

# # Re-run OLS regression with standardized features
# X_standardized = filtered_log_data_encoded[['LOG_COST', 'LOG_COST_SQ'] +
#                                            [col for col in filtered_log_data_encoded.columns
#                                             if col.startswith(('ATL_OR_DR_', 'CAMPAIGN_TYPE_', 'CHANNEL_'))]]
# X_standardized = sm.add_constant(X_standardized)  # Add constant for OLS

# y_standardized = filtered_log_data_encoded['LOG_FREE_TRIALS']

# # Fit OLS regression with standardized features
# ols_standardized_model = sm.OLS(y_standardized, X_standardized).fit()

# # Display the updated summary
# ols_standardized_summary = ols_standardized_model.summary()
# ols_standardized_summary